# ML Summer School - Assignment 4

In this assignment, we will be clustering movies on the basis of their synopsis. Unlike previous assignments, this time you will be implementing the KMeans clustering algorithm from scratch.

### Load Libraries

In [28]:
import numpy as np
import pandas as pd
import nltk
import re
import os
from sklearn import feature_extraction
import pickle
import scipy

### Load data
We will be using the synopsis of Top 100 movies ranked by IMDb. `titles`, `genres` and `synopsis` are lists stored as pickled files. We first load them into memory. 

Take a look at the lists we have just loaded.

In [29]:
titles = pickle.load(open('data/titles.pkl','rb'))
genres = pickle.load(open('data/genres.pkl','rb'))
synopsis = pickle.load(open('data/synopses.pkl','rb'))
print(len(titles), len(genres), len(synopsis))

100 100 100


### Preprocessing the Data

Let us take a look at some of the synopsis.

In [30]:
#print(synopsis[0])

In [31]:
#print(synopsis[1])


We can see that the synopsis contain a lot of names (Proper Nouns) and years. They are specific to the movies and do not seem to be helpful to cluster the movies. We should remove them

We will also be lemmatizing the words. [Lemmatizing](https://www.twinword.com/blog/what-is-lemmatization/ ) refers to replacing word by its base form (lemma). For example, lemmatizing `cars` gives `car`. Lemmatizing will help in clubbing all different inflections of words, eg. `loves`, `loving`, `loved` will all be lemmatized to `love`.

### Task 1

Complete the following function `preprocess_data`. It takes in text as input, and [removes proper nouns](https://stackoverflow.com/questions/39634222/is-there-a-way-to-remove-proper-nouns-from-a-sentence-using-python), non-alphabetic words, lemmatizes the data and lower cases the entire text.

In [32]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
st1=SnowballStemmer("english")
#st2= PorterStemmer("english")

def preprocess_data(text):
    """Removes proper noun, non-alphabetic words, lemmatizes the data and lower cases the entire text"""
    
    # YOUR CODE STARTS HERE
    
    tagged_sentence = nltk.tag.pos_tag(text.split()) 
    #print(tagged_sentence)
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    
    #print(' '.join(edited_sentence))
    text=' '.join(edited_sentence)
   # print(text)
   # print()
    text=(' '.join(filter(lambda x: x not in stopwords.words("english"),text.split())))
#    print(text)

    text=text.lower() 
    #re.sub("[^a-z]+", '',text)

    #text=text.replace(/[^a-zA-Z ]/g, ""));
    text=re.sub('[^a-zA-Z]+', ' ',text)
    #print(text)
    #print()
   # text=(list(filter(lambda x:stemming(x),text.split())))
    #print(text)
    #print()
    
   # text=(' '.join(text))
    stemmer=SnowballStemmer("english")
    
    text = " ".join([ stemmer.stem(kw) for kw in text.split(" ")])

        
    # YOUR CODE ENDS HERE
    
    return text

**Sample input**: preprocess_data(synopsis[0][:1000])


**Sample output**: 'edit edit edit day daughter wedding hears request role crime family son uniform introduces girlfriend family reception godson singer pleads help securing movie role dispatch consigliere influence studio head is unmoved morning wake bed head stallion day daughter wedding hears request role crime family son uniform introduces girlfriend family reception godson'



In [ ]:
 
print(preprocess_data(synopsis[0][:1000]))
#print(synopsis[0][:1000])
#stemmer.stem("requests")
#print(preprocess_data("requests"))


edit edit on day daughter s wed hear request role crime famili youngest son uniform introduc girlfriend famili sprawl recept godson popular singer plead help secur covet movi role dispatch consiglier influenc abras studio head unmov morn wake bed sever head prize stallion on day daughter s wed hear request role crime famili youngest son uniform introduc girlfriend famili sprawl recept godson


### Creating the Features

We will extract features from the synopsis using TfIdf Vectoriser, which will be used to cluster the movies. 

Read more about TfIdf Vectoriser [here](http://blog.christianperone.com/2011/09/machine-learning-text-feature-extraction-tf-idf-part-i/) and [here](http://blog.christianperone.com/2011/10/machine-learning-text-feature-extraction-tf-idf-part-ii/).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000, min_df=0.2,\
                                   stop_words='english', preprocessor=preprocess_data, \
                                   use_idf=True, ngram_range=(1, 1))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopsis)

print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000, min_df=0.2,\
#                                   stop_words='english', preprocessor=preprocess_data, \
#                                   use_idf=True, ngram_range=(1, 1))

#%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopsis)

#print(tfidf_matrix.shape)
#terms = tfidf_vectorizer.get_feature_names()

In [ ]:
tfidf_matrix

### Task 2 - Implementing KMeans

This is the most important task of this assignment because in this, you will implement your own K-Means. Fill in the function `KMeans` which will return the labels and cluster centers in the tuple `(labels,centers)` for the given feature matrix `X` and `num_clusters`. We also pass the `max_iter` parameter to run KMeans for that many iterations as it sometimes gets stuck on a local minima. You can learn more about Kmeans clustering [here](https://en.wikipedia.org/wiki/K-means_clustering) and [here](https://www.datascience.com/blog/k-means-clustering).

**Sample Input**: `[[2,1], [2,3], [8,1], [8,3]]`, `num_clusters=2`

**Sample Output**: `([0,0,1,1], [[2,2], [8,2]])`

In [ ]:
import random
def Kmeans(X, num_clusters=8, max_iter=300):

    # YOUR CODE STARTS HERE
    centers={}
    labels=[]
    j=0
    for i in random.sample(range(X.shape[0]),num_clusters):
        centers[j] = X[i]
        j=j+1
    cs = {}
    for i in range(max_iter):
            cs = {}
            labels=[]
            for i in range(num_clusters):
                cs[i] = []

            for featureset in X:
                dist = [np.linalg.norm(featureset-centers[centroid]) for centroid in centers]
                cn = dist.index(min(dist))
                cs[cn].append(featureset)
                labels.append(cn)
           

            for cn in cs:
                centers[cn] = np.average(cs[cn],axis=0)
   
    labels = np.asarray(labels)
    centers = np.asarray(list(centers.values()))
                

         
    # YOUR CODE ENDS HERE
    return (labels, centers) 

In [ ]:
#from sklearn.cluster import KMeans

#def Kmeans(X, num_clusters=8, max_iter=300):

    # YOUR CODE STARTS HERE
    
#    kmeans = KMeans(n_clusters=num_clusters, max_iter=max_iter).fit(X)
    # YOUR CODE ENDS HERE
#    labels= kmeans.labels_
#    centers=kmeans.cluster_centers_
#    return (labels, centers)

In [ ]:
Kmeans(np.array([[2,1], [2,3], [8,1], [8,3]]), num_clusters=2)

Cluster the synopsis (represented as TfIdf vectors) using Kmeans. You can play with different number of centers and maximum iterations to get different results.

In [ ]:
%time (labels, centers) = Kmeans(tfidf_matrix.todense(), num_clusters=3, max_iter=1000)
print(tfidf_matrix.shape)
print(tfidf_matrix.todense().shape)

Now we create a dataframe `frame` that stores the the clusters labels, names and genres for all the 100 movies.

In [ ]:
import pandas as pd
films = {'title': titles, 'synopsis': synopsis, 'cluster': labels, 'genre': genres}
frame = pd.DataFrame(films, index = [labels] , columns = ['title', 'cluster', 'genre'])
frame

Movie counts for a particular cluster.

In [ ]:
frame['cluster'].value_counts()

### Interpreting Results
We sort the cluster centers to get the most important terms per cluster and store it in `cluster_names`. We print them along with the movies in that cluster and look how well has KMeans worked.

In [27]:
print("Top terms per cluster:")
order_centroids = np.asarray(centers).argsort()[:, ::-1]
cluster_names = []
for i in range(order_centroids.shape[0]):
    print("Cluster %d words:" % i, end='')
    q = ""
    for ind in order_centroids[i][0][:6]:
        print(' %s' % terms[ind], end=',')
        q += str(terms[ind])
        q += " "
    cluster_names.append(q)
    print()
    print("Cluster %d titles:" % i, end='')
    for title in frame.loc[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print()
    print()

Top terms per cluster:
Cluster 0 words: met, reason, investig, ensu, tear, prevent,
Cluster 0 titles: Singin' in the Rain, Amadeus, Rocky, An American in Paris, The Good, the Bad and the Ugly, Butch Cassidy and the Sundance Kid, The Apartment, High Noon, The Deer Hunter, All Quiet on the Western Front, It Happened One Night, Midnight Cowboy, Rain Man, Tootsie, Nashville, Pulp Fiction, Yankee Doodle Dandy,

Cluster 1 words: pictur, intent, middl, hour, fli, afterward,
Cluster 1 titles: The Godfather, The Shawshank Redemption, Schindler's List, Casablanca, One Flew Over the Cuckoo's Nest, Citizen Kane, The Godfather: Part II, Psycho, Sunset Blvd., The Sound of Music, West Side Story, The Silence of the Lambs, The Bridge on the River Kwai, It's a Wonderful Life, Some Like It Hot, Gandhi, From Here to Eternity, Unforgiven, A Streetcar Named Desire, To Kill a Mockingbird, Doctor Zhivago, The Pianist, Goodfellas, The French Connection, Good Will Hunting, Terms of Endearment, Fargo, The Grape

### Plotting the Data
Now we plot the various movie clusters.
Basically we scale the multi-dimentional feature vector by applying 2 dimensional PCA. It is a technique used to visualize multi-dimensional plots in 2 dimensions. More about it [here](http://www.apnorton.com/blog/2016/12/19/Visualizing-Multidimensional-Data-in-Python/).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sklearnPCA

In [ ]:
pca = sklearnPCA(n_components=2) #2-dimensional PCA
transformed = pd.DataFrame(pca.fit_transform(tfidf_matrix.todense()))

In [ ]:
colors = ['red', 'blue', 'green', 'yellow', 'black', 'gray', 'orange', 'brown']
for i in range(len(cluster_names)):
    plt.scatter(transformed[labels == i][0], transformed[labels == i][1], label=cluster_names[i], c=colors[i])
plt.legend()
plt.show()

# And you're done!